In [1]:
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies')

import alchemy_tools as at
from geometry_euston import abc_to_hmatrix, distance_pbc
from parse_cube_files import CUBE

import numpy as np

In [2]:
def distance_MIC(pos_nuc, meshgrid, h_matrix):
    """
    calculates the distance between the position of the nucleus and the nearest image of a gridpoint
    
    pos_nuc: position of nucleus
    meshgrid: meshgrid containing x,y,z components of every gridpoint
    h_matrix: needed for calculation of MIC distance
    """
    
    distance = np.zeros((meshgrid[0].shape))
    
    for idx0 in range(meshgrid[0].shape[0]):
        for idx1 in range(meshgrid[0].shape[1]):
            for idx2 in range(meshgrid[0].shape[2]):
                distance[idx0][idx1][idx2] = distance_pbc(pos_nuc, np.array([meshgrid[0][idx0][idx1][idx2], meshgrid[1][idx0][idx1][idx2], meshgrid[2][idx0][idx1][idx2]]), h_matrix)
    
    return(distance)

In [3]:
def distance_MIC2(pos_nuc, meshgrid, h_matrix):
    """
    calculates the distance between the position of the nucleus and the nearest image of a gridpoint
    works so far only for cubic symmetry
    
    pos_nuc: position of nucleus
    meshgrid: meshgrid containing x,y,z components of every gridpoint
    h_matrix: needed for calculation of MIC distance
    :return: distance between pos_nuc and every gridpoint
    :rtype: numpy array of shape meshgrid.shape
    """
    
    hinv = np.linalg.inv(h_matrix)
    a_t = np.dot(hinv, pos_nuc)
    
    # calculate product of h_matrix and grid componentwise
    b_t_x = hinv[0][0]*meshgrid[0]
    b_t_y = hinv[1][1]*meshgrid[1]
    b_t_z = +hinv[2][2]*meshgrid[2]
    
    t_12_x = b_t_x - a_t[0]
    t_12_y = b_t_y - a_t[1]
    t_12_z = b_t_z - a_t[2]
    
    t_12_x -= np.round(t_12_x)
    t_12_y -= np.round(t_12_y)
    t_12_z -= np.round(t_12_z)
    
    x = np.power(h_matrix[0][0]*t_12_x, 2)
    y = np.power(h_matrix[1][1]*t_12_y, 2)
    z = np.power(h_matrix[2][2]*t_12_z, 2)
    
    return(np.sqrt(x+y+z))

In [4]:
cube = CUBE('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212/cube-files/ve_04.cube')

In [5]:
# abc_to_hmatrix(20, 20, 20, 90, 90, 90)
h_matrix = np.array([cube.X*cube.NX, cube.Y*cube.NY, cube.Z*cube.NZ])
h_matrix

array([[37.794575,  0.      ,  0.      ],
       [ 0.      , 37.794575,  0.      ],
       [ 0.      ,  0.      , 37.794575]])

In [6]:
hinv = np.linalg.inv(h_matrix)
a = cube.atoms[0][1:4]
a_t = np.dot(hinv, a)

In [7]:
hinv[0][0]

0.026458823786217993

In [8]:
meshgrid = cube.get_grid()

In [21]:
%timeit distances_mic = distance_MIC(cube.atoms[0][1:4], meshgrid, h_matrix)

1min 44s ± 1.75 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
distances_mic = distance_MIC(cube.atoms[0][1:4], meshgrid, h_matrix)

In [10]:
%timeit distance2 = distance_MIC2(cube.atoms[0][1:4], meshgrid, h_matrix)

250 ms ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
distance_mic2 = distance_MIC2(cube.atoms[0][1:4], meshgrid, h_matrix)

In [13]:
np.allclose(distances_mic,distance_mic2)

True